In [1]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
conn = psycopg2.connect(
    host='shopify-merchant-dump.ccm9mnr5avgs.ap-south-1.rds.amazonaws.com',
    port='5432',
    user='toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c',
    password='toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c_password',
    database='toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c_db'
)

#  AOV Conversion Analytics

In [3]:
query_1 = """select
created_at::date as created,
id as id,
customer_id as user_id,
contact_email as email,
source_name as ordered_via,
total_price as order_amount,
CASE
WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
ELSE 'Delivered'
END as order_status
from orders;"""

query_2 = """select
order_id,
name as product_name,
quantity from
order_line_item;"""

e=pd.read_sql(query_1,conn)
e.columns = ["created", "order_id", "user_id","email", "order_channel", "order_amount", "order_status"]
e['created'] = pd.to_datetime(e['created'])

f = pd.read_sql(query_2,conn)
f.columns = ["order_id", "product_name", "quantity"]
OrderSKUMap=f.groupby('order_id').agg(
        SKUs=pd.NamedAgg(column='product_name', aggfunc= 'nunique'),
        Quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
    ).reset_index()

e['OrderDate']=e['created'].dt.date
e['OrderDate']=pd.to_datetime(e['OrderDate'])
e['DeltaDaysinOrders']=(e.groupby('user_id').OrderDate.shift() - e.OrderDate).dt.days.abs()
e['DeltaOrderValues']=(e.groupby('user_id').order_amount.shift() - e.order_amount)

x1=e.groupby('user_id').agg(
        LastDate=pd.NamedAgg(column='OrderDate', aggfunc= 'max'),
        FirstDate=pd.NamedAgg(column='OrderDate', aggfunc='min')
    ).reset_index()


e2=e.merge(x1[['user_id','LastDate','FirstDate']], left_on='user_id', right_on='user_id')
e2=e2.merge(OrderSKUMap, left_on='order_id', right_on='order_id', how='left')

e2 = e2[e2['order_status'] != 'Cancelled']

e2=e2[e2['OrderDate']>='2023-01-01']
e2['TrxnRank'] = e2.groupby('user_id')['OrderDate'].rank(method='first')

e2['WeekCount']=e2['OrderDate'].dt.strftime('%y-w%U')
e2['MonthCount']=e2['OrderDate'].dt.strftime('%y-%b')

e2 = e2.sort_values(by='created')

In [4]:
def main_AOV(df, transx, transy,comp2_bin):
    
    grouped_y_trxn = df[df['TrxnRank'] <= transy]

    x_data = grouped_y_trxn[grouped_y_trxn['TrxnRank']<=transx]
    x2 = x_data.groupby('user_id').agg(
        x_Order_Value=pd.NamedAgg(column='order_amount', aggfunc='sum')
    ).reset_index()

    x1 = grouped_y_trxn.groupby(['user_id']).agg(
        xDate=pd.NamedAgg(column='created', aggfunc= lambda x: x.nlargest(2).min()),
        yDate=pd.NamedAgg(column='created', aggfunc= 'max'),
        Order_Count=pd.NamedAgg(column='order_id', aggfunc='nunique'),
        Total_Order_Value=pd.NamedAgg(column='order_amount', aggfunc='sum')
    ).reset_index()

    merged = pd.merge(x1, x2, on = 'user_id', how = 'left')

    merged['y_Order_Value'] = merged['Total_Order_Value'] - merged['x_Order_Value']
    merged['x_Order_Value'] = merged['x_Order_Value'].fillna(0)

    merged=merged.sort_values(by='xDate')

    merged['Bins']=merged['x_Order_Value'].transform(lambda x: pd.qcut(x.rank(method='first'), q = 10, labels = ['Bin1', 'Bin2', 'Bin3', 'Bin4','Bin5','Bin6','Bin7','Bin8','Bin9','Bin10']))
    merged['Converted'] = 0
    merged.loc[merged['Order_Count']>transx,'Converted'] = 1
    merged['xDate'] = pd.to_datetime(merged['xDate'])

    convert_df = merged.groupby('Bins').agg(
        Acquired_Pool=pd.NamedAgg(column='user_id', aggfunc= 'nunique'),
        Repeat_Pool=pd.NamedAgg(column='Converted', aggfunc='sum'),
        Order_Value=pd.NamedAgg(column='Total_Order_Value', aggfunc='sum'),
        Bin_Average_Order_Value=pd.NamedAgg(column='x_Order_Value', aggfunc='mean')
    ).reset_index()

    convert_df['Conversion_Rate'] = convert_df['Repeat_Pool']/convert_df['Acquired_Pool']

    comp1=convert_df.copy()

    comp2=merged[merged['Bins']==comp2_bin][['user_id','x_Order_Value']]
    comp2=comp2.rename(columns={'x_Order_Value':'Order_Value'})
    comp2=comp2.set_index('user_id')

    convert_df_sort = convert_df.sort_values(by='Conversion_Rate', ascending=False)
    if(transx==1):
        bins_to_return = convert_df_sort['Bins'].head(3).tolist()
    elif(transx==2):
        bins_to_return = convert_df_sort['Bins'].head(3).tolist()
    else:
        bins_to_return = convert_df_sort['Bins'].head(7).tolist()

    return comp1,bins_to_return,comp2

In [5]:
conv_analytics_new_to_repeat_AOV, one_to_two_trxn_AOV_bin, conv_analytics_new_to_repeat_AOV_2 = main_AOV(e2,1,2,'Bin1')
conv_analytics_two_to_three_AOV, two_to_three_trxn_AOV_bin, conv_analytics_two_to_three_AOV_2 = main_AOV(e2,2,3,'Bin2')
conv_analytics_three_to_four_AOV, three_to_four_trxn_AOV_bin, conv_analytics_three_to_four_AOV_2 = main_AOV(e2,3,4,'Bin3')

In [6]:
conv_analytics_new_to_repeat_AOV_2

,Order_Value
user_id,
6.406576e+12,176.0
6.413898e+12,196.0
6.414255e+12,176.0
6.414823e+12,196.0
6.414602e+12,196.0
...,...
6.898378e+12,199.0
6.898373e+12,199.0
6.898294e+12,199.0


In [7]:
# Conversion Analytics based on AOV:
# NEW_REPEAT:
# Component 1: conv_analytics_new_to_repeat_AOV
# Component 2: conv_analytics_new_to_repeat_AOV_2

# 2_3:
# Component 1: conv_analytics_two_to_three_AOV
# Component 2: conv_analytics_two_to_three_AOV_2

# 3_4:
# Component 1: conv_analytics_three_to_four_AOV
# Component 2: conv_analytics_three_to_four_AOV_2

# Pincode Conversion Analytics

In [8]:
query_3 = '''
SELECT order_id AS order_id,
zip AS pincode,
city as city
FROM order_billing_address;
'''
x = pd.read_sql(query_3, conn)
x = x[x['pincode'].str.len()==6]
x = x[x['pincode']!='695 01']
x = x[x['pincode']!='609.60']
x = x[x['pincode']!= 'V1M3S8']

query_4 = '''select
    created_at::date as OrderDate,
    id as order_id,
    customer_id as user_id,
    CASE
        WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
        ELSE 'Delivered'
    END as order_status
    from orders;'''

query_5 = '''select
    name as product_name,
    order_id,
    product_id,
    quantity,
    price
    from order_line_item;'''

a=pd.read_sql(query_4,conn)
b = pd.read_sql(query_5,conn)

a['orderdate'] = pd.to_datetime(a['orderdate'])
# a = a[a['orderdate']>='2023-09-01']

a = a[a['order_status']=='Delivered']
a.drop('order_status',axis=1,inplace=True)
a['TrxnRank'] = a.groupby(['user_id'])['orderdate'].rank(method='first')

merged_new = pd.merge(a,b,on='order_id')
final_pincode = pd.merge(merged_new,x,on='order_id',how='right')
final_pincode.dropna(inplace=True)

In [9]:
def main_pincode(df, transx, transy,comp2_bin):
    trans_x = df[df['TrxnRank']==transx]
    trans_y_users = trans_x['user_id'].unique()
    trans_y = df[(df['TrxnRank']==transy) & (df['user_id'].isin(trans_y_users))]

    xuser=trans_x.groupby(['pincode','user_id']).agg(
        x_user=pd.NamedAgg(column='user_id', aggfunc='nunique'),
    )
    xuser_count=xuser.groupby('pincode').agg(
        x_trxn_pool=pd.NamedAgg(column='x_user', aggfunc='sum'),
    ).reset_index()

    yuser=trans_y.groupby(['pincode','user_id']).agg(
        y_user=pd.NamedAgg(column='user_id', aggfunc='nunique'),
    )
    yuser_count=yuser.groupby('pincode').agg(
        y_trxn_pool=pd.NamedAgg(column='y_user', aggfunc='sum'),
    ).reset_index()

    m=pd.merge(xuser, yuser, left_index=True, right_index=True, how='inner').reset_index()
    yuser_count=m.groupby('pincode').agg(
        y_trxn_pool=pd.NamedAgg(column='y_user', aggfunc='sum')
    ).reset_index()

    main_grouped_for_pincode = pd.merge(xuser_count, yuser_count, on='pincode', how='left').fillna(0)
    main_grouped_for_pincode['Conversion_Rate']=main_grouped_for_pincode['y_trxn_pool']/main_grouped_for_pincode['x_trxn_pool']

    main_grouped_for_pincode['Bins']=main_grouped_for_pincode['Conversion_Rate'].transform(lambda x: pd.qcut(x.rank(method='first'), q = 5, labels = ['Bin1', 'Bin2', 'Bin3', 'Bin4', 'Bin5']))

    df_city=df[['pincode','city']]
    main_grouped_for_pincode=pd.merge(main_grouped_for_pincode,df_city,on='pincode',how='left')

    convert_df = main_grouped_for_pincode.groupby('Bins').agg(
        Acquired_Pool=pd.NamedAgg(column='x_trxn_pool', aggfunc= 'sum'),
        Repeat_Pool=pd.NamedAgg(column='y_trxn_pool', aggfunc='sum'),
        Conversion_Rate=pd.NamedAgg(column='Conversion_Rate', aggfunc='mean'),
        no_of_pincodes=pd.NamedAgg(column='pincode', aggfunc='nunique')
    ).reset_index()

    comp1=convert_df

    comp2=main_grouped_for_pincode[main_grouped_for_pincode['Bins']==comp2_bin]
    comp2=comp2.rename(columns={'x_trxn_pool':'Acquired_Pool', 'y_trxn_pool':'Repeat_Pool'})
    
    # f=(main_grouped_for_pincode['Bin']=='B4')

    if ((transx == 1) & (transx==2)):
        pot_pincode = main_grouped_for_pincode[(main_grouped_for_pincode['Bins']=='Bin5')]['pincode'].to_list()
    else:
        bins_to_include = ['Bin2','Bin3', 'Bin4', 'Bin5']
        pot_pincode = main_grouped_for_pincode[main_grouped_for_pincode['Bins'].isin(bins_to_include)]['pincode'].to_list()

    return comp1, pot_pincode, comp2

In [10]:
conv_analytics_new_to_repeat_pincode, one_to_two_trxn_pincode, conv_analytics_new_to_repeat_pincode_2 = main_pincode(final_pincode,1,2,'Bin4')
conv_analytics_two_to_three_pincode, two_to_three_trxn_pincode, conv_analytics_two_to_three_pincode_2 = main_pincode(final_pincode,2,3,'Bin4')
conv_analytics_three_to_four_pincode, three_to_four_trxn_pincode, conv_analytics_three_to_four_pincode_2 = main_pincode(final_pincode,3,4,'Bin4')

In [11]:
conv_analytics_three_to_four_pincode_2

,pincode,Acquired_Pool,Repeat_Pool,Conversion_Rate,Bins,city
60,110002,1,1.0,1.0,Bin4,Delhi
61,110002,1,1.0,1.0,Bin4,Delhi
62,110002,1,1.0,1.0,Bin4,Delhi
63,110002,1,1.0,1.0,Bin4,New Delhi
64,110002,1,1.0,1.0,Bin4,Delhi
...,...,...,...,...,...,...
64007,845401,2,1.0,0.5,Bin4,East Champaran
64008,845401,2,1.0,0.5,Bin4,East Champaran
64009,845401,2,1.0,0.5,Bin4,East Champaran
64010,845401,2,1.0,0.5,Bin4,East Champaran


In [12]:
# Conversion Analytics based on Pincode:
# NEW_REPEAT:
# Component 1: conv_analytics_new_to_repeat_pincode
# Component 2: conv_analytics_new_to_repeat_pincode_2

# 2_3:
# Component 1: conv_analytics_two_to_three_pincode
# Component 2: conv_analytics_two_to_three_pincode_2

# 3_4:
# Component 1: conv_analytics_three_to_four_pincode
# Component 2: conv_analytics_three_to_four_pincode_2

# SKU Conversion Analytics

In [13]:
query_6 = '''select
    created_at::date as OrderDate,
    id as order_id,
    customer_id as user_id,
    CASE
        WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
        ELSE 'Delivered'
    END as order_status
    from orders;'''

query_7 = '''select
    name as product_name,
    order_id,
    product_id,
    quantity,
    price,
    variant_title as variant
    from order_line_item;'''

query_8 = '''select
    id as product_id,
    title as product_title,
    product_type as product_category,
    handle,
    tags
    from products;'''

In [14]:
ee = pd.read_sql(query_6, conn)
ff = pd.read_sql(query_7, conn)
gg = pd.read_sql(query_8, conn)

ee = ee[ee['order_status']=='Delivered']
ee.drop('order_status',axis=1,inplace=True)

ee_sorted = ee.sort_values(by=['user_id', 'orderdate', 'order_id'])
merged_df = pd.merge(ff, ee, on='order_id')

merged_df.dropna(inplace=True)

merged_df['Transaction_Rank'] = merged_df.groupby(['user_id', 'product_name'])['orderdate'].rank(method='first')

final = pd.merge(merged_df, gg, on='product_id')

final['Total Order Value'] = final['quantity'] * final['price']

In [15]:
def main_x(final, transactionx, transactiony, parameter):

    final1 = final.copy()
    
    if parameter == "tags":
        final_tags = final1.copy()
        final_tags['tags'] = final_tags['tags'].str.split(',')
        final_tags = final_tags.explode('tags', ignore_index=True)
        final1 = final_tags
    else:
        pass
    
    trans_x = final1[final1['Transaction_Rank']==transactionx]
    trans_y = final1[(final1['Transaction_Rank']==transactiony)]

    xuser_count = trans_x.groupby(parameter).agg(
        x_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique')
    ).reset_index()

    yuser_count = trans_y.groupby(parameter).agg(
        y_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique')
    ).reset_index()
        
    main_grouped_for_title = pd.merge(xuser_count, yuser_count, on=parameter, how='left')
    main_grouped_for_title['y_trxn_pool'].fillna(0, inplace=True)
    
    result = main_grouped_for_title.copy()
    
    result['Conversion_Rate_x_to_y'] = result['y_trxn_pool']/result['x_trxn_pool']
    result['Conversion_Rate_x_to_y'].fillna(0, inplace= True)  
    result['Cart_Penetration_for_x_transaction'] = result['x_trxn_pool']/(result['x_trxn_pool'].sum())
    result['Cart_Penetration_for_y_transaction'] = result['y_trxn_pool']/(result['y_trxn_pool'].sum())
        
    result1 = result.copy()
    result2 = result.copy()
    result1 = result1.sort_values(by='Conversion_Rate_x_to_y', ascending=False)
    result2 = result2.sort_values(by='Cart_Penetration_for_x_transaction', ascending=False)

    result1['Mix']= result1['Conversion_Rate_x_to_y']*100/result1['Conversion_Rate_x_to_y'].sum()
    result1['CumuSum']= result1['Mix'].cumsum()
    result1['Conversion Rate Bin']=0
    if(transactionx==1):
        result1.loc[result1['CumuSum']>=60,"Conversion Rate Bin"]='HighConversion'
        result1.loc[(result1['CumuSum']<60),"Conversion Rate Bin"]='LowConversion'
    elif(transactionx==2):
        result1.loc[result1['CumuSum']>=95,"Conversion Rate Bin"]='HighConversion'
        result1.loc[(result1['CumuSum']<95),"Conversion Rate Bin"]='LowConversion'        
    else:
        result1.loc[result1['CumuSum']>=30,"Conversion Rate Bin"]='HighConversion'
        result1.loc[(result1['CumuSum']<30) ,"Conversion Rate Bin"]='LowConversion'  
    result2['Mix']= result2['Cart_Penetration_for_x_transaction']*100/result2['Cart_Penetration_for_x_transaction'].sum()
    result2['CumuSum']= result2['Mix'].cumsum()
    result2['Cart Penetration Bin']=0
    result2.loc[result2['CumuSum']<=75,"Cart Penetration Bin"]='HighCartPenetration'
    result2.loc[(result2['CumuSum']>75) & (result2['CumuSum']<=101),"Cart Penetration Bin"]='LowCartPenetration'

    result2 = result2[[parameter,'Cart Penetration Bin']]
    final_merged_on_title_x_to_y = pd.merge(result1, result2, on=parameter)

    #-------------------------------------------------------------------------------------------------------------------------------

    final_merged_on_title_x_to_y = final_merged_on_title_x_to_y[final_merged_on_title_x_to_y['Conversion Rate Bin'] != 0]

    #-------------------------------------------------------------------------------------------------------------------------------

    final_merged_on_title_x_to_y['Bin'] = final_merged_on_title_x_to_y['Conversion Rate Bin'] + '_' + final_merged_on_title_x_to_y['Cart Penetration Bin']
    # final_merged_on_title_x_to_y.drop(['Conversion Rate Bin','Cart Penetration Bin'], axis=1, inplace= True)

    #-------------------------------------------------------------------------------------------------------------------------------

    merged_title_on_bin = final_merged_on_title_x_to_y.groupby('Bin').agg({
        parameter: 'nunique'
    }).reset_index()

    #-------------------------------------------------------------------------------------------------------------------------------

    x = final_merged_on_title_x_to_y.columns
    y = []
    for i in x:
        if "x_" in i:
            i = i.replace("x_", f"{str(transactionx)}_")
        if "_x" in i:
            i = i.replace("_x", f"_{str(transactiony)}")
        if "y_" in i:
            i = i.replace("y_", f"{str(transactiony)}_")
        if "_y" in i:
            i = i.replace("_y", f"_{str(transactiony)}")
        y.append(i)
    final_merged_on_title_x_to_y.columns = y

    return final_merged_on_title_x_to_y, result1[[parameter, "Conversion Rate Bin"]], final_merged_on_title_x_to_y[[parameter, f"Conversion_Rate_{transactionx}_to_{transactiony}", f"Cart_Penetration_for_{transactionx}_transaction", f"Cart_Penetration_for_{transactiony}_transaction"]]

In [16]:
prdt_page, input, prdt_page_2  = main_x(final, 1, 2, "product_name")
one_to_two_trxn_prod = input[input['Conversion Rate Bin']=='HighConversion']['product_name'].to_list()
prdt_page1, input2,prdt_page1_2 = main_x(final, 2 ,3, 'product_name')
two_to_three_trxn_prod = input2[input2['Conversion Rate Bin']=='HighConversion']['product_name'].to_list()
prdt_page2,input3,prdt_page2_2 = main_x(final, 3, 4, 'product_name')
three_to_four_trxn_prod = input3[input3['Conversion Rate Bin']=='HighConversion']['product_name'].to_list()

In [17]:
def component1(final, transactionx, transactiony, parameter):
    prdt_page_x, input, prdt_page_x_2 = main_x(final, transactionx, transactiony, parameter)
    component1_dump = pd.merge(final, prdt_page_x, on='product_name', how='left')
    acq = f"{transactionx}_trxn_pool"
    repeat = f"{transactiony}_trxn_pool"
    conversion = f"Conversion_Rate_{transactionx}_to_{transactiony}"
    cart_pen_1 = f"Cart_Penetration_for_{transactionx}_transaction"
    cart_pen_2 = f"Cart_Penetration_for_{transactiony}_transaction"
    component1_to_show = component1_dump.groupby("Bin").agg(
        Product_Counts = pd.NamedAgg(column = 'product_name', aggfunc = 'nunique'),
        Quantity = pd.NamedAgg(column = 'quantity', aggfunc = 'sum'),
        Acquired_Pool = pd.NamedAgg(column = acq, aggfunc = 'sum'),
        Repeat_Pool = pd.NamedAgg(column = repeat, aggfunc = 'sum'),
        Total_Order_Value = pd.NamedAgg(column = "Total Order Value", aggfunc = 'sum'),
        rename1 = pd.NamedAgg(column = conversion, aggfunc = 'mean'),
        rename2 = pd.NamedAgg(column = cart_pen_1, aggfunc = 'mean'),
        rename3 = pd.NamedAgg(column = cart_pen_2, aggfunc = 'mean')).reset_index()
    component1_to_show.rename(columns = {"rename1": conversion, "rename2": cart_pen_1, "rename3":cart_pen_2}, inplace = True)
    component1_to_show.set_index("Bin", inplace = True)
    return component1_to_show, component1_dump

In [18]:
component_1_to_show, dump = component1(final, 3, 4, 'product_name')

In [19]:
#Component 2
component_1_to_show

,Product_Counts,Quantity,Acquired_Pool,Repeat_Pool,Total_Order_Value,Conversion_Rate_3_to_4,Cart_Penetration_for_3_transaction,Cart_Penetration_for_4_transaction
Bin,,,,,,,,
HighConversion_HighCartPenetration,52,57508,4770854.0,1570146.0,31117656.0,0.392952,0.051360,0.041255
HighConversion_LowCartPenetration,149,21452,70926.0,23152.0,17317590.0,0.272846,0.002049,0.001632
LowConversion_LowCartPenetration,26,1727,2362.0,2362.0,1554769.0,1.000000,0.000844,0.002061


In [20]:
def component2(parameter, Bin):
    df = prdt_page[prdt_page["Bin"] == Bin]
    df.set_index(parameter, inplace = True)
    df.drop(columns = ["Mix", "CumuSum"], inplace = True)
    return df

In [21]:
#Component 2
component2('product_name', "LowConversion_LowCartPenetration")

,1_trxn_pool,2_trxn_pool,Conversion_Rate_1_to_2,Cart_Penetration_for_1_transaction,Cart_Penetration_for_2_transaction,Conversion Rate Bin,Cart Penetration Bin,Bin
product_name,,,,,,,,
Holiday Blend Coffee - French Roast - Espresso / 250 gm,1,1.0,1.000000,0.000014,0.000142,LowConversion,LowCartPenetration,LowConversion_LowCartPenetration
Annanas: PineApple Fermented Coffee - Espresso / 1000 gm,1,1.0,1.000000,0.000014,0.000142,LowConversion,LowCartPenetration,LowConversion_LowCartPenetration
Thogarihunkal Estate Coffee - Moka Pot / 1000 gm,1,1.0,1.000000,0.000014,0.000142,LowConversion,LowCartPenetration,LowConversion_LowCartPenetration
Chemex Coffee Maker (Wooden Collar) - Wooden Collar / 6 Cups,1,1.0,1.000000,0.000014,0.000142,LowConversion,LowCartPenetration,LowConversion_LowCartPenetration
Ethiopia Coffee (From Yirgacheffe) - Syphon Coffee / 250 gm,1,1.0,1.000000,0.000014,0.000142,LowConversion,LowCartPenetration,LowConversion_LowCartPenetration
...,...,...,...,...,...,...,...,...
Thippanahalli Estate Coffee - Espresso / 1000 gm,6,1.0,0.166667,0.000082,0.000142,LowConversion,LowCartPenetration,LowConversion_LowCartPenetration
Blueberry Mocha - Speciality Blend - 500 gm / Cold Brew,6,1.0,0.166667,0.000082,0.000142,LowConversion,LowCartPenetration,LowConversion_LowCartPenetration
Hario Filter-In Coffee Bottle - Chocolate Brown,12,2.0,0.166667,0.000163,0.000283,LowConversion,LowCartPenetration,LowConversion_LowCartPenetration


# Transaction Dump

In [22]:
t = pd.merge(e2,final_pincode,on='order_id',how='left')
t = t[(t['user_id_x'] == t['user_id_y']) & (t['TrxnRank_x'] == t['TrxnRank_y'])]
t = t[['created', 'order_amount', 'user_id_x', 'Quantity', 'TrxnRank_x', 'product_id', 'product_name', 'pincode','city','order_id','email']]
t.rename(columns = {'user_id_x':'user_id', 'TrxnRank_x':'TrxnRank'}, inplace = True)
t.dropna(inplace=True)
t

,created,order_amount,user_id,Quantity,TrxnRank,product_id,product_name,pincode,city,order_id,email
2,2023-01-01,1040.0,6.412658e+12,4,1.0,6.613557e+12,Easy Brew (Hot Brew) - Baba Budangiri Estate C...,482001,Jabalpur,4951038394506,agrawalanurag1970@gmail.com
3,2023-01-01,1040.0,6.412658e+12,4,1.0,6.607820e+12,Easy Brew (Hot Brew) - Mysore Nuggets - Pack o...,482001,Jabalpur,4951038394506,agrawalanurag1970@gmail.com
4,2023-01-01,1040.0,6.412658e+12,4,1.0,6.613470e+12,Easy Brew (Hot Brew) - Tangerine: Orange Ferme...,482001,Jabalpur,4951038394506,agrawalanurag1970@gmail.com
5,2023-01-01,1040.0,6.412658e+12,4,1.0,6.612165e+12,Easy Brew (Hot Brew) - Whiskey Barrel Aged Cof...,482001,Jabalpur,4951038394506,agrawalanurag1970@gmail.com
6,2023-01-01,220.0,6.413857e+12,1,1.0,6.613470e+12,Easy Brew (Hot Brew) - Tangerine: Orange Ferme...,400612,Thane,4950780739722,arfatk882866@gmail.com
...,...,...,...,...,...,...,...,...,...,...,...
49871,2024-01-02,199.0,6.980956e+12,1,1.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,841301,Chhapra,5376149422218,khanshamimkhan78@gmail.com
49872,2024-01-03,299.0,6.977607e+12,1,1.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,803110,Chandi,5376707920010,ramjipdsinha01@gmail.com
49873,2024-01-03,199.0,6.979764e+12,1,1.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,686012,KOTTAYAM,5376906821770,leelamathomas@gmail.com
49874,2024-01-03,324.0,6.705760e+12,1,2.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,431005,Chatrapati sambhajinagar,5377011712138,druttampanchal1959@gmail.com


In [23]:
t = t[t['TrxnRank']<=4.0]

In [24]:
t.groupby('order_id').agg(
    AOV=pd.NamedAgg(column='order_amount', aggfunc='sum')
)

,AOV
order_id,
4950457811082,1008.0
4950752166026,2450.0
4950774677642,2160.0
4950780575882,650.0
4950780739722,220.0
...,...
5376349569162,299.0
5376707920010,299.0
5376906821770,199.0


In [25]:
dump_AOV = t.groupby(['user_id','TrxnRank']).agg(
    AOV=pd.NamedAgg(column='order_amount', aggfunc='sum')
).reset_index()
dump_AOV

,user_id,TrxnRank,AOV
0,3.555623e+12,1.0,840.0
1,3.571057e+12,1.0,368.0
2,3.591889e+12,1.0,630.0
3,3.591889e+12,2.0,2550.0
4,3.595168e+12,1.0,1300.0
...,...,...,...
32508,6.980984e+12,1.0,299.0
32509,6.980991e+12,1.0,299.0
32510,6.981047e+12,1.0,224.0
32511,6.981178e+12,1.0,324.0


In [26]:
dump=pd.merge(dump_AOV, t, on =['user_id','TrxnRank'], how='left').dropna()
dump

,user_id,TrxnRank,AOV,created,order_amount,Quantity,product_id,product_name,pincode,city,order_id,email
0,3.555623e+12,1.0,840.0,2023-01-11,840.0,1,7.532732e+12,Brazil Cerrado Coffee (Latin America) - Whole ...,600126,Kanchipuram,4961785151626,abhishek_vit@hotmail.com
1,3.571057e+12,1.0,368.0,2023-07-05,368.0,1,6.747180e+12,Hazelnut Cold Brew - Easy Brew Cold Brew Bags ...,462016,Korba,5143186178186,connectvinayak@gmail.com
2,3.591889e+12,1.0,630.0,2023-03-20,630.0,1,7.532732e+12,Brazil Cerrado Coffee (Latin America) - French...,700032,Kolkata,5038845952138,bshoumo@gmail.com
3,3.591889e+12,2.0,2550.0,2023-07-07,1275.0,2,7.644293e+12,Ethiopia Coffee (From Yirgacheffe) - French Pr...,700032,Kolkata,5145122504842,bshoumo@gmail.com
4,3.591889e+12,2.0,2550.0,2023-07-07,1275.0,2,6.559076e+12,Enamel Coffee Mug - Quantity - 1,700032,Kolkata,5145122504842,bshoumo@gmail.com
...,...,...,...,...,...,...,...,...,...,...,...,...
40260,6.980984e+12,1.0,299.0,2024-01-02,299.0,1,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,500059,Hyderabad,5376167215242,tlnarasimham@yahoo.co.in
40261,6.980991e+12,1.0,299.0,2024-01-02,299.0,1,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,670703,Iritty,5376171573386,pramodpunnad@gmail.com
40262,6.981047e+12,1.0,224.0,2024-01-02,224.0,1,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,682012,Kochi,5376203587722,samueldrake107@gmail.com
40263,6.981178e+12,1.0,324.0,2024-01-02,324.0,1,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,140125,Ropar,5376288882826,sunilkumar@gmail.com


In [27]:
dump['Bins1']=dump[dump['TrxnRank']==1.0]['AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = 10, labels = ['B1', 'B2', 'B3', 'B4','B5','B6','B7','B8','B9','B10']))
dump['Bins2']=dump[dump['TrxnRank']==2.0]['AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = 10, labels = ['B1', 'B2', 'B3', 'B4','B5','B6','B7','B8','B9','B10']))
dump['Bins3']=dump[dump['TrxnRank']==3.0]['AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = 10, labels = ['B1', 'B2', 'B3', 'B4','B5','B6','B7','B8','B9','B10']))

In [28]:
t=dump.copy()

In [29]:
t['one_to_two_pin']=0
t['two_to_three_pin']=0
t['three_to_four_pin']=0
t.loc[(t['pincode'].isin(one_to_two_trxn_pincode)) & (t['TrxnRank']==1.0),'one_to_two_pin'] = 1
t.loc[(t['pincode'].isin(two_to_three_trxn_pincode)) & (t['TrxnRank']==2.0),'two_to_three_pin'] = 1
t.loc[(t['pincode'].isin(three_to_four_trxn_pincode)) & (t['TrxnRank']==3.0),'three_to_four_pin'] = 1

In [30]:
t['one_to_two_bin']=0
t['two_to_three_bin']=0
t['three_to_four_bin']=0
t.loc[(t['Bins1'].isin(one_to_two_trxn_AOV_bin)) & (t['TrxnRank']==1.0),'one_to_two_bin'] = 1
t.loc[(t['Bins2'].isin(two_to_three_trxn_AOV_bin)) & (t['TrxnRank']==2.0),'two_to_three_bin'] = 1
t.loc[(t['Bins3'].isin(three_to_four_trxn_AOV_bin)) & (t['TrxnRank']==3.0),'three_to_four_bin'] = 1

In [31]:
t['one_to_two_prod']=0
t['two_to_three_prod']=0
t['three_to_four_prod']=0
t.loc[(t['product_name'].isin(one_to_two_trxn_prod)) & (t['TrxnRank']==1.0),'one_to_two_prod'] = 1
t.loc[(t['product_name'].isin(two_to_three_trxn_prod)) & (t['TrxnRank']==2.0),'two_to_three_prod'] = 1
t.loc[(t['product_name'].isin(three_to_four_trxn_prod)) & (t['TrxnRank']==3.0),'three_to_four_prod'] = 1

In [32]:
t_upd = t.groupby('user_id').agg(
    order_count=pd.NamedAgg(column='order_id', aggfunc='nunique')
).reset_index()
t_upd

,user_id,order_count
0,3.555623e+12,1
1,3.571057e+12,1
2,3.591889e+12,2
3,3.595168e+12,2
4,3.607596e+12,3
...,...,...
26451,6.980984e+12,1
26452,6.980991e+12,1
26453,6.981047e+12,1
26454,6.981178e+12,1


In [33]:
t=pd.merge(t,t_upd, on='user_id',how='left')
t

,user_id,TrxnRank,AOV,created,order_amount,Quantity,product_id,product_name,pincode,city,...,one_to_two_pin,two_to_three_pin,three_to_four_pin,one_to_two_bin,two_to_three_bin,three_to_four_bin,one_to_two_prod,two_to_three_prod,three_to_four_prod,order_count
0,3.555623e+12,1.0,840.0,2023-01-11,840.0,1,7.532732e+12,Brazil Cerrado Coffee (Latin America) - Whole ...,600126,Kanchipuram,...,1,0,0,0,0,0,0,0,0,1
1,3.571057e+12,1.0,368.0,2023-07-05,368.0,1,6.747180e+12,Hazelnut Cold Brew - Easy Brew Cold Brew Bags ...,462016,Korba,...,1,0,0,0,0,0,1,0,0,1
2,3.591889e+12,1.0,630.0,2023-03-20,630.0,1,7.532732e+12,Brazil Cerrado Coffee (Latin America) - French...,700032,Kolkata,...,1,0,0,0,0,0,1,0,0,2
3,3.591889e+12,2.0,2550.0,2023-07-07,1275.0,2,7.644293e+12,Ethiopia Coffee (From Yirgacheffe) - French Pr...,700032,Kolkata,...,0,1,0,0,0,0,0,1,0,2
4,3.591889e+12,2.0,2550.0,2023-07-07,1275.0,2,6.559076e+12,Enamel Coffee Mug - Quantity - 1,700032,Kolkata,...,0,1,0,0,0,0,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40260,6.980984e+12,1.0,299.0,2024-01-02,299.0,1,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,500059,Hyderabad,...,1,0,0,0,0,0,1,0,0,1
40261,6.980991e+12,1.0,299.0,2024-01-02,299.0,1,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,670703,Iritty,...,1,0,0,0,0,0,1,0,0,1
40262,6.981047e+12,1.0,224.0,2024-01-02,224.0,1,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,682012,Kochi,...,1,0,0,0,0,0,1,0,0,1
40263,6.981178e+12,1.0,324.0,2024-01-02,324.0,1,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,140125,Ropar,...,0,0,0,0,0,0,1,0,0,1


In [34]:
t[t['user_id']==6908212379786]

,user_id,TrxnRank,AOV,created,order_amount,Quantity,product_id,product_name,pincode,city,...,one_to_two_pin,two_to_three_pin,three_to_four_pin,one_to_two_bin,two_to_three_bin,three_to_four_bin,one_to_two_prod,two_to_three_prod,three_to_four_prod,order_count
37185,6.908212e+12,1.0,324.0,2023-11-23,324.0,1,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,201014,Ghaziabad,...,1,0,0,0,0,0,1,0,0,2
37186,6.908212e+12,2.0,623.0,2023-12-12,623.0,2,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,110092,Delhi,...,0,1,0,0,0,0,0,0,0,2


In [35]:
t_x=t[['user_id','email','city','TrxnRank']]
t_x_1=t_x[t_x['TrxnRank']==1]
t_x_2=t_x[t_x['TrxnRank']==2]
t_x_3=t_x[t_x['TrxnRank']==3]
t_x_1

,user_id,email,city,TrxnRank
0,3.555623e+12,abhishek_vit@hotmail.com,Kanchipuram,1.0
1,3.571057e+12,connectvinayak@gmail.com,Korba,1.0
2,3.591889e+12,bshoumo@gmail.com,Kolkata,1.0
5,3.595168e+12,dcdchaudhuri@gmail.com,Kalyani,1.0
6,3.595168e+12,dcdchaudhuri@gmail.com,Kalyani,1.0
...,...,...,...,...
40260,6.980984e+12,tlnarasimham@yahoo.co.in,Hyderabad,1.0
40261,6.980991e+12,pramodpunnad@gmail.com,Iritty,1.0
40262,6.981047e+12,samueldrake107@gmail.com,Kochi,1.0
40263,6.981178e+12,sunilkumar@gmail.com,Ropar,1.0


# Conversion, Potential, DropOff Counts

In [36]:
main_dump = t.groupby('user_id').agg(
    FirstDate=pd.NamedAgg(column='created', aggfunc='min'),
    LastDate=pd.NamedAgg(column='created', aggfunc='max'),
    AOV=pd.NamedAgg(column='order_amount', aggfunc='sum'),
    orderCount=pd.NamedAgg(column='order_id', aggfunc='nunique'),
    pincodeList=pd.NamedAgg(column='pincode', aggfunc=lambda x: x.unique()),
    productName=pd.NamedAgg(column='product_name', aggfunc=lambda x: x.unique())
).reset_index()
main_dump

,user_id,FirstDate,LastDate,AOV,orderCount,pincodeList,productName
0,3.555623e+12,2023-01-11,2023-01-11,840.0,1,[600126],[Brazil Cerrado Coffee (Latin America) - Whole...
1,3.571057e+12,2023-07-05,2023-07-05,368.0,1,[462016],[Hazelnut Cold Brew - Easy Brew Cold Brew Bags...
2,3.591889e+12,2023-03-20,2023-07-07,3180.0,2,[700032],[Brazil Cerrado Coffee (Latin America) - Frenc...
3,3.595168e+12,2023-07-26,2023-12-12,7096.0,2,[741235],"[Holiday Blend Coffee - Whole Beans / 250 gm, ..."
4,3.607596e+12,2023-03-24,2023-08-06,4358.0,3,"[560070, 560082]",[Easy Brew (Hot Brew) - Whiskey Barrel Aged Co...
...,...,...,...,...,...,...,...
26451,6.980984e+12,2024-01-02,2024-01-02,299.0,1,[500059],[South Indian Traditional Filter Coffee - Dark...
26452,6.980991e+12,2024-01-02,2024-01-02,299.0,1,[670703],[South Indian Traditional Filter Coffee - Dark...
26453,6.981047e+12,2024-01-02,2024-01-02,224.0,1,[682012],[South Indian Traditional Filter Coffee - Dark...
26454,6.981178e+12,2024-01-02,2024-01-02,324.0,1,[140125],[South Indian Traditional Filter Coffee - Dark...


In [37]:
main_dump["PinCode_Tag_1_to_2"] = 0
for i in range (len(main_dump)):
    main_dump.iloc[i, -1] = len([x for x in main_dump.iloc[i, 5] if (x) in one_to_two_trxn_pincode])


main_dump["prod_name_1_to_2"] = 0
for i in range (len(main_dump)):
    main_dump.iloc[i, -1] = len([x for x in main_dump.iloc[i, 6] if (x) in one_to_two_trxn_prod])

main_dump['AOV_bin_1_to_2'] = 0
main_dump.loc[main_dump['orderCount']==1,'AOV_bin_1_to_2']= main_dump.loc[main_dump['orderCount']==1]['AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = 10, labels = ['B1','B2','B3','B4','B5','B6','B7','B8','B9','B10']))#.astype('int64')

In [38]:
main_dump["PinCode_Tag_2_to_3"] = 0
for i in range (len(main_dump)):
    main_dump.iloc[i, -1] = len([x for x in main_dump.iloc[i, 5] if (x) in two_to_three_trxn_pincode])

main_dump["prod_name_2_to_3"] = 0
for i in range (len(main_dump)):
    main_dump.iloc[i, -1] = len([x for x in main_dump.iloc[i, 6] if (x) in two_to_three_trxn_prod])

main_dump['AOV_bin_2_to_3'] = 0
main_dump.loc[main_dump['orderCount']==2,'AOV_bin_2_to_3']= main_dump.loc[main_dump['orderCount']==2]['AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = 10, labels = ['B1','B2','B3','B4','B5','B6','B7','B8','B9','B10']))#.astype('int64')

In [39]:
main_dump["PinCode_Tag_3_to_4"] = 0
for i in range (len(main_dump)):
    main_dump.iloc[i, -1] = len([x for x in main_dump.iloc[i, 5] if (x) in three_to_four_trxn_pincode])

main_dump["prod_name_3_to_4"] = 0
for i in range (len(main_dump)):
    main_dump.iloc[i, -1] = len([x for x in main_dump.iloc[i, 6] if (x) in three_to_four_trxn_prod])

main_dump['AOV_bin_3_to_4'] = 0
main_dump.loc[main_dump['orderCount']==3,'AOV_bin_3_to_4']= main_dump.loc[main_dump['orderCount']==3]['AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = 10, labels = ['Bin1','Bin2','Bin3','Bin4','Bin5','Bin6','Bin7','Bin8','Bin9','Bin10']))#.astype('int64')

In [40]:
main_dump['AOV_check_1']=0
main_dump.loc[((main_dump['orderCount']==1) & (main_dump['AOV_bin_1_to_2'].isin(one_to_two_trxn_AOV_bin))), 'AOV_check_1']=1
main_dump['AOV_check_2']=0
main_dump.loc[((main_dump['orderCount']==2) & (main_dump['AOV_bin_2_to_3'].isin(two_to_three_trxn_AOV_bin))), 'AOV_check_2']=1
main_dump['AOV_check_3']=0
main_dump.loc[((main_dump['orderCount']==3) & (main_dump['AOV_bin_3_to_4'].isin(three_to_four_trxn_AOV_bin))), 'AOV_check_3']=1

In [41]:
main_dump['FirstDate'] = pd.to_datetime(main_dump['FirstDate'])
three_months_ago = datetime.now() - timedelta(days=3 * 30)
three_months_ago_date = three_months_ago.date()
three_months_ago_date=pd.to_datetime(three_months_ago_date)


In [42]:
main_dump['Potential_1_to_2'] = 0
main_dump.loc[((main_dump['orderCount']==1) & (main_dump['FirstDate']>three_months_ago_date) & ((main_dump['PinCode_Tag_1_to_2']>1) | (main_dump['AOV_check_1']==1) | (main_dump['prod_name_1_to_2']>0))),'Potential_1_to_2']= 1

In [43]:
main_dump['Potential_2_to_3'] = 0
main_dump.loc[((main_dump['orderCount']==2) & (main_dump['FirstDate']>three_months_ago_date) & ((main_dump['PinCode_Tag_2_to_3']>1) | (main_dump['AOV_check_2']==1) | (main_dump['prod_name_2_to_3']>0))),'Potential_2_to_3']= 1

In [44]:
main_dump[main_dump['prod_name_2_to_3']>0]

,user_id,FirstDate,LastDate,AOV,orderCount,pincodeList,productName,PinCode_Tag_1_to_2,prod_name_1_to_2,AOV_bin_1_to_2,...,prod_name_2_to_3,AOV_bin_2_to_3,PinCode_Tag_3_to_4,prod_name_3_to_4,AOV_bin_3_to_4,AOV_check_1,AOV_check_2,AOV_check_3,Potential_1_to_2,Potential_2_to_3
1,3.571057e+12,2023-07-05,2023-07-05,368.0,1,[462016],[Hazelnut Cold Brew - Easy Brew Cold Brew Bags...,1,1,B7,...,1,0,0,1,0,0,0,0,0,0
2,3.591889e+12,2023-03-20,2023-07-07,3180.0,2,[700032],[Brazil Cerrado Coffee (Latin America) - Frenc...,1,3,0,...,3,B9,1,2,0,0,0,0,0,0
3,3.595168e+12,2023-07-26,2023-12-12,7096.0,2,[741235],"[Holiday Blend Coffee - Whole Beans / 250 gm, ...",1,4,0,...,1,B10,0,5,0,0,0,0,0,0
4,3.607596e+12,2023-03-24,2023-08-06,4358.0,3,"[560070, 560082]",[Easy Brew (Hot Brew) - Whiskey Barrel Aged Co...,2,5,0,...,3,0,2,4,Bin9,0,0,1,0,0
5,3.628984e+12,2023-07-26,2023-07-26,5781.0,1,[600040],[South Indian Traditional Filter Coffee - Dark...,1,3,B10,...,2,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26442,6.980962e+12,2024-01-02,2024-01-02,224.0,1,[689532],[South Indian Traditional Filter Coffee - Dark...,1,1,B4,...,1,0,0,1,0,0,0,0,1,0
26446,6.980971e+12,2024-01-02,2024-01-02,224.0,1,[799001],[South Indian Traditional Filter Coffee - Dark...,1,1,B4,...,1,0,0,1,0,0,0,0,1,0
26447,6.980974e+12,2024-01-02,2024-01-02,224.0,1,[695586],[South Indian Traditional Filter Coffee - Dark...,1,1,B4,...,1,0,0,1,0,0,0,0,1,0
26450,6.980982e+12,2024-01-02,2024-01-02,224.0,1,[743134],[South Indian Traditional Filter Coffee - Dark...,1,1,B4,...,1,0,0,1,0,0,0,0,1,0


In [45]:
main_dump['Potential_3_to_4'] = 0
main_dump.loc[((main_dump['orderCount']==3) & (main_dump['FirstDate']>three_months_ago_date) & ((main_dump['PinCode_Tag_3_to_4']>1) | (main_dump['AOV_check_3']==1))),'Potential_3_to_4']= 1

In [46]:
main_dump[main_dump['AOV_check_3']==1]

,user_id,FirstDate,LastDate,AOV,orderCount,pincodeList,productName,PinCode_Tag_1_to_2,prod_name_1_to_2,AOV_bin_1_to_2,...,AOV_bin_2_to_3,PinCode_Tag_3_to_4,prod_name_3_to_4,AOV_bin_3_to_4,AOV_check_1,AOV_check_2,AOV_check_3,Potential_1_to_2,Potential_2_to_3,Potential_3_to_4
4,3.607596e+12,2023-03-24,2023-08-06,4358.0,3,"[560070, 560082]",[Easy Brew (Hot Brew) - Whiskey Barrel Aged Co...,2,5,0,...,0,2,4,Bin9,0,0,1,0,0,0
19,3.714291e+12,2023-06-20,2023-09-12,9926.5,3,"[110057, 160009]",[Mullayangiri Estate Coffee - 250 gm / Aeropre...,2,6,0,...,0,2,5,Bin10,0,0,1,0,0,0
26,5.057566e+12,2023-07-26,2023-09-11,3860.0,3,[411060],"[Holiday Blend Coffee - Whole Beans / 250 gm, ...",1,5,0,...,0,1,6,Bin8,0,0,1,0,0,0
42,5.131501e+12,2023-05-15,2023-10-16,7720.0,3,[305901],[Thippanahalli Estate Coffee - Espresso / 250 ...,1,7,0,...,0,0,7,Bin10,0,0,1,0,0,0
63,5.166361e+12,2023-05-21,2023-07-20,5330.0,3,[482001],[Hazelnut Cold Brew - Easy Brew Cold Brew Bags...,1,5,0,...,0,1,5,Bin9,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25221,6.931869e+12,2023-12-06,2023-12-09,872.0,3,[500089],[South Indian Traditional Filter Coffee - Dark...,1,2,0,...,0,1,2,Bin3,0,0,1,0,0,1
25569,6.938955e+12,2023-12-10,2023-12-10,747.0,3,[382030],[South Indian Traditional Filter Coffee - Dark...,1,2,0,...,0,0,2,Bin1,0,0,1,0,0,1
25607,6.939399e+12,2023-12-10,2023-12-22,772.0,3,[700014],[South Indian Traditional Filter Coffee - Dark...,1,2,0,...,0,1,2,Bin2,0,0,1,0,0,1
25702,6.943113e+12,2023-12-12,2023-12-23,872.0,3,[302016],[South Indian Traditional Filter Coffee - Dark...,1,2,0,...,0,0,2,Bin3,0,0,1,0,0,1


In [47]:
main_dump[main_dump['Potential_3_to_4']==1]#.sum()

,user_id,FirstDate,LastDate,AOV,orderCount,pincodeList,productName,PinCode_Tag_1_to_2,prod_name_1_to_2,AOV_bin_1_to_2,...,AOV_bin_2_to_3,PinCode_Tag_3_to_4,prod_name_3_to_4,AOV_bin_3_to_4,AOV_check_1,AOV_check_2,AOV_check_3,Potential_1_to_2,Potential_2_to_3,Potential_3_to_4
108,5.326260e+12,2023-10-26,2023-10-28,5110.0,3,[560102],[Single Malt Whiskey Barrel Aged Coffee - Whol...,1,5,0,...,0,1,5,Bin9,0,0,1,0,0,1
9076,6.653446e+12,2023-11-01,2023-12-21,897.0,3,[533004],[South Indian Traditional Filter Coffee - Dark...,1,1,0,...,0,1,1,Bin3,0,0,1,0,0,1
12062,6.693872e+12,2023-10-10,2023-12-15,4112.0,3,[122001],[Thippanahalli Estate Coffee - Pour Over / 250...,1,4,0,...,0,1,5,Bin8,0,0,1,0,0,1
13326,6.703546e+12,2023-11-02,2023-11-13,847.0,3,[493118],[South Indian Traditional Filter Coffee - Dark...,1,2,0,...,0,0,2,Bin2,0,0,1,0,0,1
17010,6.740588e+12,2023-12-10,2023-12-27,872.0,3,[671348],[South Indian Traditional Filter Coffee - Dark...,1,2,0,...,0,1,2,Bin2,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25221,6.931869e+12,2023-12-06,2023-12-09,872.0,3,[500089],[South Indian Traditional Filter Coffee - Dark...,1,2,0,...,0,1,2,Bin3,0,0,1,0,0,1
25569,6.938955e+12,2023-12-10,2023-12-10,747.0,3,[382030],[South Indian Traditional Filter Coffee - Dark...,1,2,0,...,0,0,2,Bin1,0,0,1,0,0,1
25607,6.939399e+12,2023-12-10,2023-12-22,772.0,3,[700014],[South Indian Traditional Filter Coffee - Dark...,1,2,0,...,0,1,2,Bin2,0,0,1,0,0,1
25702,6.943113e+12,2023-12-12,2023-12-23,872.0,3,[302016],[South Indian Traditional Filter Coffee - Dark...,1,2,0,...,0,0,2,Bin3,0,0,1,0,0,1


In [48]:
main_dump['Convert_1_to_2'] = 0
main_dump['Convert_2_to_3'] = 0
main_dump['Convert_3_to_4'] = 0

main_dump.loc[(main_dump['orderCount']==2), 'Convert_1_to_2']= 1
main_dump.loc[(main_dump['orderCount']==3), 'Convert_2_to_3']= 1
main_dump.loc[(main_dump['orderCount']==4), 'Convert_3_to_4']= 1

In [49]:
main_dump['WeekCount'] = main_dump['FirstDate'].dt.strftime('%Y-w%U')

In [50]:
main_dump['Inactive'] = 0
main_dump.loc[(main_dump['Convert_2_to_3']!=1) & (main_dump['Potential_2_to_3']!=1) & (main_dump['Convert_3_to_4']!=1) & (main_dump['Potential_3_to_4']!=1) & (main_dump['FirstDate']>three_months_ago_date),'Inactive'] = 1

In [51]:
main_dump['Churned'] = 0
main_dump.loc[(main_dump['Convert_1_to_2']!=1) & (main_dump['Potential_1_to_2']!=1) & (main_dump['FirstDate']<three_months_ago_date),'Churned'] = 1

In [52]:
component_1 = main_dump.groupby('WeekCount').agg(
    Acquired_User_Pool=pd.NamedAgg(column='user_id', aggfunc='nunique'),
    Second_Order_Completed_User_Pool=pd.NamedAgg(column='Convert_1_to_2', aggfunc='sum'),
    Potential_for_2nd_Order=pd.NamedAgg(column='Potential_1_to_2', aggfunc='sum'),
    Third_Order_Completed_User_Pool=pd.NamedAgg(column='Convert_2_to_3', aggfunc='sum'),
    Potential_for_3rd_Order=pd.NamedAgg(column='Potential_2_to_3', aggfunc='sum'),
    Fourth_Order_Completed_User_Pool=pd.NamedAgg(column='Convert_3_to_4', aggfunc='sum'),
    Potential_for_4th_Order=pd.NamedAgg(column='Potential_3_to_4', aggfunc='sum'),
).reset_index()
component_1['Churned_Cust_Pool']=component_1['Acquired_User_Pool']-component_1['Second_Order_Completed_User_Pool']-component_1['Potential_for_2nd_Order']
component_1['Inactive_Cust_Pool']=component_1['Second_Order_Completed_User_Pool']-component_1['Potential_for_3rd_Order'] - component_1['Third_Order_Completed_User_Pool'] + component_1['Third_Order_Completed_User_Pool'] -component_1['Fourth_Order_Completed_User_Pool'] - component_1['Potential_for_4th_Order']
component_1=component_1.tail(12)
component_1

,WeekCount,Acquired_User_Pool,Second_Order_Completed_User_Pool,Potential_for_2nd_Order,Third_Order_Completed_User_Pool,Potential_for_3rd_Order,Fourth_Order_Completed_User_Pool,Potential_for_4th_Order,Churned_Cust_Pool,Inactive_Cust_Pool
42,2023-w43,545,51,475,11,29,4,7,19,11
43,2023-w44,1098,121,939,21,59,13,15,38,34
44,2023-w45,825,95,706,15,35,8,6,24,46
45,2023-w46,715,70,627,10,38,6,7,18,19
46,2023-w47,861,84,753,13,54,8,11,24,11
47,2023-w48,633,72,543,11,37,5,8,18,22
48,2023-w49,781,59,705,9,28,4,4,17,23
49,2023-w50,494,47,436,8,28,3,5,11,11
50,2023-w51,168,10,158,0,6,0,0,0,4
51,2023-w52,207,6,198,2,4,0,0,3,2


In [53]:
# component_1.to_csv('Conversion_Analytics.csv')

In [54]:
#Lists
Acquired_List = main_dump[main_dump['orderCount']==1][['user_id','FirstDate','pincodeList']]
Acquired_List=pd.merge(Acquired_List,t_x_1,on='user_id',how='left')
Acquired_List.drop_duplicates(subset=['user_id'], keep='first', inplace=True, ignore_index=True)
Acquired_List.drop(columns='TrxnRank', inplace=True)
Acquired_List=Acquired_List.rename(columns= {'FirstDate' : 'Date'})
# Acquired_List.to_csv('Acquired_List.csv')

Second_Order_Completed_User_List = main_dump[main_dump['Convert_1_to_2']==1][['user_id','FirstDate','pincodeList']]
Second_Order_Completed_User_List=pd.merge(Second_Order_Completed_User_List,t_x_1,on='user_id',how='left')
Second_Order_Completed_User_List.drop_duplicates(subset=['user_id'], keep='first', inplace=True, ignore_index=True)
Second_Order_Completed_User_List.drop(columns='TrxnRank', inplace=True)
Second_Order_Completed_User_List=Second_Order_Completed_User_List.rename(columns= {'FirstDate' : 'Date'})
# Second_Order_Completed_User_List.to_csv('Second_Order_Completed_User_List.csv')

Third_Order_Completed_User_List = main_dump[main_dump['Convert_2_to_3']==1][['user_id','FirstDate','pincodeList']]
Third_Order_Completed_User_List=pd.merge(Third_Order_Completed_User_List,t_x_2,on='user_id',how='left')
Third_Order_Completed_User_List.drop_duplicates(subset=['user_id'], keep='first', inplace=True, ignore_index=True)
Third_Order_Completed_User_List.drop(columns='TrxnRank', inplace=True)
Third_Order_Completed_User_List=Third_Order_Completed_User_List.rename(columns= {'FirstDate' : 'Date'})
# Third_Order_Completed_User_List.to_csv('Third_Order_Completed_User_List.csv')

Fourth_Order_Completed_User_List = main_dump[main_dump['Convert_3_to_4']==1][['user_id','FirstDate','pincodeList']]
Fourth_Order_Completed_User_List=pd.merge(Fourth_Order_Completed_User_List,t_x_3,on='user_id',how='left')
Fourth_Order_Completed_User_List.drop_duplicates(subset=['user_id'], keep='first', inplace=True, ignore_index=True)
Fourth_Order_Completed_User_List.drop(columns='TrxnRank', inplace=True)
Fourth_Order_Completed_User_List=Fourth_Order_Completed_User_List.rename(columns= {'FirstDate' : 'Date'})
# Fourth_Order_Completed_User_List.to_csv('Fourth_Order_Completed_User_List.csv')

Potential_for_2nd_Order_List = main_dump[main_dump['Potential_1_to_2']==1][['user_id','FirstDate','pincodeList']]
Potential_for_2nd_Order_List=pd.merge(Potential_for_2nd_Order_List,t_x_1,on='user_id',how='left')
Potential_for_2nd_Order_List.drop_duplicates(subset=['user_id'], keep='first', inplace=True, ignore_index=True)
Potential_for_2nd_Order_List.drop(columns='TrxnRank', inplace=True)
Potential_for_2nd_Order_List=Potential_for_2nd_Order_List.rename(columns= {'FirstDate' : 'Date'})
# Potential_for_2nd_Order_List.to_csv('Potential_for_2nd_Order_List.csv')

Potential_for_3rd_Order_List = main_dump[main_dump['Potential_2_to_3']==1][['user_id','FirstDate','pincodeList']]
Potential_for_3rd_Order_List=pd.merge(Potential_for_3rd_Order_List,t_x_2,on='user_id',how='left')
Potential_for_3rd_Order_List.drop_duplicates(subset=['user_id'], keep='first', inplace=True, ignore_index=True)
Potential_for_3rd_Order_List.drop(columns='TrxnRank', inplace=True)
Potential_for_3rd_Order_List=Potential_for_3rd_Order_List.rename(columns= {'FirstDate' : 'Date'})
# Potential_for_3rd_Order_List.to_csv('Potential_for_3rd_Order_List.csv')

Potential_for_4th_Order_List = main_dump[main_dump['Potential_3_to_4']==1][['user_id','FirstDate','pincodeList']]
Potential_for_4th_Order_List=pd.merge(Potential_for_4th_Order_List,t_x_3,on='user_id',how='left')
Potential_for_4th_Order_List.drop_duplicates(subset=['user_id'], keep='first', inplace=True, ignore_index=True)
Potential_for_4th_Order_List.drop(columns='TrxnRank', inplace=True)
Potential_for_4th_Order_List=Potential_for_4th_Order_List.rename(columns= {'FirstDate' : 'Date'})
# Potential_for_4th_Order_List.to_csv('Potential_for_4th_Order_List.csv')

Churned_Cust_List = main_dump[main_dump['Churned']==1][['user_id','FirstDate','pincodeList']]
Churned_Cust_List=pd.merge(Churned_Cust_List,t_x_1,on='user_id',how='left')
Churned_Cust_List.drop_duplicates(subset=['user_id'], keep='first', inplace=True, ignore_index=True)
Churned_Cust_List.drop(columns='TrxnRank', inplace=True)
Churned_Cust_List=Churned_Cust_List.rename(columns= {'FirstDate' : 'Date'})
# Churned_Cust_List.to_csv('Churned_Cust_List.csv')

Inactive_Cust_List = main_dump[main_dump['Inactive']==1][['user_id','FirstDate','pincodeList']]
Inactive_Cust_List=pd.merge(Inactive_Cust_List,t_x,on='user_id',how='left')
Inactive_Cust_List.drop(columns='TrxnRank',inplace=True)
Inactive_Cust_List.drop_duplicates(subset=['user_id'], keep='first', inplace=True, ignore_index=True)
# Inactive_Cust_List.to_csv('Inactive_Cust_List.csv')

Inactive_Cust_List

,user_id,FirstDate,pincodeList,email,city
0,3.663657e+12,2023-11-24,[401303],mehul19851@gmail.com,Virar
1,3.678238e+12,2023-12-15,[395007],kshah.1602@gmail.com,Surat
2,3.741123e+12,2023-10-19,[403507],info@primepropertiesgoa.com,Mapusa
3,5.044019e+12,2023-11-24,[400005],meha.chaturvedi03@gmail.com,Mumbai
4,5.082156e+12,2023-12-18,[700030],jitaghosh69@gmail.com,Kolkata
...,...,...,...,...,...
7228,6.980984e+12,2024-01-02,[500059],tlnarasimham@yahoo.co.in,Hyderabad
7229,6.980991e+12,2024-01-02,[670703],pramodpunnad@gmail.com,Iritty
7230,6.981047e+12,2024-01-02,[682012],samueldrake107@gmail.com,Kochi
7231,6.981178e+12,2024-01-02,[140125],sunilkumar@gmail.com,Ropar
